# LangChain4j API Overview

In [1]:
%useLatestDescriptors
%use coroutines
%use langchain4j

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

In [3]:
val model = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .build()

## Chat API

In [4]:
model.chat("How do I create in Kotlin a plus function for a Point data class that adds two Points together in the form point1 + point2?")

In Kotlin, you can overload the `plus` operator for a data class by defining the `plus` function. For your `Point` data class, you can create a function that takes another `Point` as a parameter and returns a new `Point` that represents the sum of the two points.

Here’s how you can do it:

1. Define your `Point` data class with `x` and `y` properties.
2. Implement the `plus` operator function to add two `Point` instances.

Here’s an example implementation:

```kotlin
data class Point(val x: Int, val y: Int) {
    operator fun plus(other: Point): Point {
        return Point(this.x + other.x, this.y + other.y)
    }
}

fun main() {
    val point1 = Point(1, 2)
    val point2 = Point(3, 4)

    val result = point1 + point2
    println(result) // Output: Point(x=4, y=6)
}
```

### Explanation:
- The `data class Point` defines a simple point with `x` and `y` coordinates.
- The `operator fun plus(other: Point): Point` function is defined to overload the `+` operator. It takes another `Poin

## Prompts

### 1 API

In [5]:
import dev.langchain4j.data.message.SystemMessage.systemMessage
import dev.langchain4j.data.message.UserMessage.userMessage

val messages = buildList {
    add(systemMessage(
        """
            You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """.trimIndent()
    ))
    add(userMessage(
        """
            What foldRight does? What will be the intermidieate and the end result of the foldRight function call?
            ```kotlin
                "baba".foldRight("") { x, acc -> "${'$'}x${'$'}{acc}${'$'}{acc}" })
            ```
        """.trimIndent()
    ))
}
val response = model.chat(messages).aiMessage()
response

AiMessage { text = "In Kotlin, the `foldRight` function is a higher-order function that processes a collection or a string from the last element to the first. It takes an initial accumulator value and a lambda function that combines each element with the accumulator.

In your example, the `foldRight` function is called on the string `"baba"` with an initial accumulator value of `""` (an empty string). The lambda function takes two parameters: `x` (the current character from the string) and `acc` (the accumulated result).

Here's how the `foldRight` operation works step-by-step for the string `"baba"`:

1. Start with the last character `'a'`:
   - `x = 'a'`, `acc = ""`
   - Result: `"a$acc$acc"` → `"a"`

2. Move to the second last character `'b'`:
   - `x = 'b'`, `acc = "a"`
   - Result: `"b$a$a"` → `"bab"`

3. Move to the third last character `'a'`:
   - `x = 'a'`, `acc = "bab"`
   - Result: `"a$bab$bab"` → `"ababab"`

4. Move to the first character `'b'`:
   - `x = 'b'`, `acc = "ababa

### 2 API

In [6]:
interface AssistantFirst {
    fun chat(userMessage: String): String
}

In [7]:
import dev.langchain4j.service.AiServices

val aiService = AiServices
    .builder(AssistantFirst::class.java)
    .chatModel(model)
    .systemMessageProvider {
        """
        You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """.trimIndent()
    }
    .build()

In [8]:
aiService.chat("""
Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
Make sure to include all imports, but don’t use star imports.
""".trimIndent())

Here's a simple Kotlin program that creates a flow of five elements (1, 2, 3, 4, and 5) and prints each element using the `collect` function. The code follows idiomatic Kotlin conventions and includes all necessary imports.

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    createFlow().collect { value ->
        println(value)
    }
}

fun createFlow(): Flow<Int> = flow {
    for (i in 1..5) {
        emit(i)
    }
}
```

### Explanation:
- **Imports**: Only the necessary classes are imported for clarity.
- **`main` function**: Uses `runBlocking` to create a coroutine scope and collect the flow.
- **`createFlow` function**: Generates a flow of integers from 1 to 5 using the `flow` builder and emits each value.
- **`collect`**: The `collect` function is called with a lambda to print each emitted value.

### 3 API

In [9]:
import dev.langchain4j.service.SystemMessage

interface AssistantSecond {

    @SystemMessage(
        """
        You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """
    )
    fun chat(userMessage: String): String
}

In [10]:
val assistant = AiServices.create(AssistantSecond::class.java, model)

In [11]:
assistant.chat("""
Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
Make sure to include all imports, but don’t use star imports.
""".trimIndent())

Here's a concise and idiomatic Kotlin code snippet that creates a flow of five elements and prints each one using the `collect` function:

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking

fun main() = runBlocking {
    createFlow().collect { value ->
        println(value)
    }
}

fun createFlow(): Flow<Int> = flow {
    for (i in 1..5) {
        emit(i)
    }
}
```

### Explanation:
- We use `runBlocking` to create a coroutine scope for the main function.
- The `createFlow` function generates a flow of integers from 1 to 5 using the `flow` builder.
- The `collect` function is called to print each emitted value.

## Stream

### Async

In [12]:
runBlocking {
    val messages = buildList {
        add(systemMessage("You are an expert Kotlin developer."))
        add(
            userMessage(
                """
                Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
                Make sure to include all imports, but don’t use star imports.
            """.trimIndent()
            )
        )
    }
    model.chat(messages)
}


ChatResponse { aiMessage = AiMessage { text = "Certainly! Below is a Kotlin program that creates a flow of five elements (1, 2, 3, 4, and 5) and prints each one of them using the `collect` function without any arguments. 

Make sure you have the necessary dependencies for Kotlin Coroutines in your project to run this code.

```kotlin
import kotlinx.coroutines.CoroutineScope
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.launch
import kotlinx.coroutines.runBlocking

fun main() = runBlocking {
    val numberFlow: Flow<Int> = flow {
        for (i in 1..5) {
            emit(i) // Emit numbers 1 to 5
        }
    }

    // Collecting the flow
    numberFlow.collect { value ->
        println(value) // Print each emitted value
    }
}
```

### Explanation:
- We import necessary classes from the `kotlinx.coroutines` and `kotlinx.coroutines.flow` packages.
- We define a `Flow<Int>` that emits numbers fr

### Flow

In [13]:
@file:DependsOn("dev.langchain4j:langchain4j-kotlin:1.9.1-beta17")

In [14]:
import dev.langchain4j.kotlin.model.chat.StreamingChatModelReply
import dev.langchain4j.kotlin.model.chat.chatFlow
import dev.langchain4j.model.openai.OpenAiStreamingChatModel
import kotlinx.coroutines.flow.Flow

val streamModel = OpenAiStreamingChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .build()

val flowResponse = streamModel.chatFlow {
    messages += systemMessage("You are an expert Kotlin developer.")
    messages += userMessage(
        """
            Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
            Make sure to include all imports, but don’t use star imports.
        """.trimIndent()
    )
}

runBlocking {
    flowResponse.collect {
        when (it) {
            is StreamingChatModelReply.PartialResponse -> {
                print(it.partialResponse) // Stream output as it arrives
            }

            is StreamingChatModelReply.CompleteResponse -> {
                println("\nComplete: ${it.response.aiMessage().text()}")
            }

            is StreamingChatModelReply.Error -> {
                println("Error occurred: ${it.cause.message}")
            }
        }
    }
}

Certainly! Below is an example of a Kotlin program that creates a flow of five elements (1, 2, 3, 4, and 5) and prints each element using the `collect` function. The necessary imports are included without using star imports.

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    val numberFlow: Flow<Int> = flow {
        for (i in 1..5) {
            emit(i)
        }
    }

    numberFlow.collect()
}
```

In this code:

- We import the necessary classes from the `kotlinx.coroutines.flow` package.
- We define a `Flow<Int>` that emits the numbers 1 through 5.
- We use `runBlocking` to create a coroutine scope and call `collect()` on the flow to print each element. 

To print each element, you can modify the `collect` function as follows:

```kotlin
numberFlow.collect { value ->
    println(value)
}
```

This will ensure that each emitted value is print

### AiService

In [15]:
@file:DependsOn("dev.langchain4j:langchain4j-reactor:1.9.1-beta17")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactor:1.10.2")

In [16]:
import reactor.core.publisher.Flux

interface AssistantThird {
    fun chat(message: String): Flux<String>
}

In [17]:
val assistant = AiServices.create(AssistantThird::class.java, streamModel)

In [18]:
import kotlinx.coroutines.reactive.asFlow

val response = assistant.chat("""
Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
Make sure to include all imports, but don’t use star imports.
""".trimIndent()).asFlow()

runBlocking {
    response.collect {
        print(it)
    }
}

To create a flow of five elements (1, 2, 3, 4, and 5) and print each one of them using the `collect` function in Kotlin, you can follow the example below. This example assumes you are using Kotlin's coroutines and flow features.

Here's the complete code:

```kotlin
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    // Create a flow of five elements
    val numbersFlow: Flow<Int> = flow {
        for (i in 1..5) {
            emit(i) // Emit each number
        }
    }

    // Collect and print each element from the flow
    numbersFlow.collect { number ->
        println(number)
    }
}
```

### Explanation:
- We import necessary classes from the `kotlinx.coroutines` library.
- The `main` function is defined as a coroutine using `runBlocking`, which allows us to call suspend functions.
- We create a `Flow` of integers u

## Structured Outputs

### JSON_MODE

In [19]:
import com.fasterxml.jackson.annotation.JsonCreator
import com.fasterxml.jackson.annotation.JsonProperty

/**
 * Represents a person with basic personal attributes.
 *
 * @property name The name of the person.
 * @property age The age of the person in years.
 * @property height The height of the person in meters.
 * @property married Indicates if the person is married.
 */
data class Person @JsonCreator constructor(
    @JsonProperty("name") val name: String,
    @JsonProperty("age") val age: Int,
    @JsonProperty("height") val height: Double,
    @JsonProperty("married") val married: Boolean
)

interface PersonExtractor {
    fun extractPersonFrom(text: String): Person
}

val jsonModel = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .responseFormat("json_schema")
    .strictJsonSchema(true)
    .build()

val personExtractor = AiServices.create(PersonExtractor::class.java, jsonModel)

In [20]:
personExtractor.extractPersonFrom(
    """
        John is 42 years old and lives an independent life.
        He stands 1.75 meters tall and carries himself with confidence.
        Currently unmarried, he enjoys the freedom to focus on his personal goals and interests.
    """
)

Person(name=John, age=42, height=1.75, married=false)

In [21]:
data class Step @JsonCreator constructor(
    @JsonProperty("explanation") val explanation: String,
    @JsonProperty("output") val output: String
)

data class MathResponse @JsonCreator constructor(
    @JsonProperty("steps") val steps: List<Step>,
    @JsonProperty("finalAnswer") val finalAnswer: String
)

interface MathAssistant {
    @SystemMessage("You are a helpful math tutor.")
    fun chat(message: String): MathResponse
}

val assistant = AiServices.create(MathAssistant::class.java, jsonModel)


In [22]:
val response = assistant.chat("solve 8x + 31 = 2")

println(response.steps.withIndex().joinToString("\n") { "${it.index + 1}. ${it.value.explanation}" })
println("Answer:")
println(response.finalAnswer)

1. Subtract 31 from both sides to isolate the term with x.
2. This simplifies to 8x = -29.
3. Now, divide both sides by 8 to solve for x.
4. This can be simplified to x = -3.625 or left as a fraction.
Answer:
x = -3.625 or x = -29/8


### Function calling

In [23]:
import dev.langchain4j.agent.tool.Tool

class Tools {
    @Tool
    fun plus(a: Int, b: Int): Int = a + b

    @Tool
    fun minus(a: Int, b: Int): Int = a - b

    @Tool
    fun times(a: Int, b: Int): Int = a * b

    @Tool
    fun div(a: Int, b: Int): Int = a / b
}

interface MathAssistant {
    @SystemMessage("You are a helpful math tutor.")
    fun chat(message: String): String
}

val assistant = AiServices.builder(MathAssistant::class.java)
    .chatModel(model)
    .tools(Tools())
    .build()

In [24]:
assistant.chat("""
Solve this:
* (5 + 10) * (20 - 3) / 4
* (a + b)^2
* ax^2 + bx + c = 0 for a=1, b=-3, c=-4
*  S = n * (a1 + an) / 2
    n = 10
    a1 = 3
    an = 20
""")

Let's break down the calculations step by step:

1. **Calculate (5 + 10) * (20 - 3) / 4**:
   - \(5 + 10 = 15\)
   - \(20 - 3 = 17\)
   - \(15 * 17 = 255\)
   - \(255 / 4 = 63.75\)

2. **Calculate (a + b)^2**:
   - This expression cannot be evaluated without specific values for \(a\) and \(b\).

3. **Solve \(ax^2 + bx + c = 0\) for \(a=1\), \(b=-3\), \(c=-4\)**:
   - The equation becomes \(1x^2 - 3x - 4 = 0\).
   - The roots can be calculated using the quadratic formula:
     \[
     x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
     \]
   - Here, \(b^2 - 4ac = (-3)^2 - 4(1)(-4) = 9 + 16 = 25\).
   - Thus, \(x = \frac{3 \pm 5}{2}\).
   - The roots are \(x = 4\) and \(x = -1\).

4. **Calculate \(S = n * (a1 + an) / 2\)**:
   - \(n = 10\), \(a1 = 3\), \(an = 20\).
   - \(S = 10 * (3 + 20) / 2 = 10 * 23 / 2 = 115\).

### Summary of Results:
1. **(5 + 10) * (20 - 3) / 4 = 63.75**
2. **(a + b)^2 = Not enough information**
3. **Roots of \(x^2 - 3x - 4 = 0\) are \(x = 4\) and \(x = -1\)**
4. **S = 1